In [1]:
%reload_ext autoreload
%autoreload 2

import random
import numpy as np
from matplotlib import pyplot as plt
from syd import make_viewer

from vrAnalysis.database import get_database
from vrAnalysis.sessions import B2Session
from vrAnalysis.processors import SpkmapProcessor
from dimilibi import Population

sessiondb = get_database("vrSessions")

In [2]:
"""
Let's think about what I need here.

1. All the regression models need to be able to compare the same target neurons, with the same cross-validated timepoints. For the RBF(POS) model, I actually need train/train/val/test splits. 
So, I'd like a "registry" which will create a "Population" object with the correct time splits 
for all sessions and save it somewhere. Then, I can just load it from the registry.
- So I'll save the population data in a designated folder, under a repr of the session name. 
- Each time I try to analyze a session, I'll check the registry to see if the population data exists,
if not I'll make a new one and save it. 

"""

print('bad docstring! no docstrign!')

bad docstring! no docstrign!


In [ ]:
from dataclasses import dataclass

@dataclass
class RegistryParameters:
    time_split_prms: dict
    cell_split_prms: dict
    registry_path: str

class PopulationRegistry:
    def __init__(self, registry_path: str):
        self.registry_path = registry_path

    def _check_population(self, session: B2Session):
        pass

    def _load_population(self, session: B2Session):
        pass

    def _save_population(self, session: B2Session, population: Population):
        pass
    
    def _make_population(self, session: B2Session):
        pass

    def get_population(self, session: B2Session):
        pass

In [4]:
ises = sessiondb.iter_sessions(imaging=True)
ses = random.choice(ises)
print(ses)

B2Session(mouse_name='ATL022', date='2023-04-05', session_id='702')


In [5]:
time_split_prms = dict(
    num_groups=4,
    relative_size=[4, 4, 1, 1],
    chunks_per_group=25,
)
npop = Population(ses.spks.T, time_split_prms=time_split_prms)

In [7]:
source, target = npop.get_split_data(time_idx=[0, 1])
print(source.shape, target.shape)

torch.Size([8726, 11503]) torch.Size([8725, 11503])
